In [7]:
import sys
import torch
import numpy as np
sys.path.append('..')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf
from erank.regularization import EffectiveRankRegularization
import matplotlib.pyplot as plt

In [8]:
def erank_np(matrix_A: np.ndarray):
    s = np.linalg.svd(matrix_A, compute_uv=False)
    probs = s / s.sum()
    entropy = (- probs * np.log(probs)).sum()
    return np.exp(entropy)

In [9]:
def erank_torch(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
    """Calculates the effective rank of a matrix.
    Args:
        matrix_A (torch.Tensor): Matrix of shape m x n. 
        center_matrix_A (bool): Center the matrix 
    Returns:
        torch.Tensor: Effective rank of matrix_A
    """
    assert matrix_A.ndim == 2
    _, s, _ = torch.pca_lowrank(matrix_A, center=center_matrix_A, niter=1, q=min(matrix_A.shape[0], matrix_A.shape[1]))
    # normalizes input s -> scale independent!
    return torch.exp(torch.distributions.Categorical(s).entropy())

In [35]:
matrix_A = np.random.normal(size=(2,3))
v = np.random.normal(size=(1,3))
matrix_A

array([[ 0.27748275,  0.29311095,  0.33319707],
       [-0.02326997,  0.27713841, -1.31620778]])

In [32]:
erank_np(matrix_A), erank_torch(torch.from_numpy(matrix_A))

(1.8610527199527092, tensor(1.8611, dtype=torch.float64))

In [33]:
cov_A = np.cov(matrix_A)

In [34]:
erank_np(cov_A), np.linalg.matrix_rank(cov_A)

(1.9290512871992447, 2)